In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2, l1



In [12]:
path = "proccesed_dataset"

In [13]:
# Define path to your dataset (replace with actual path if needed)
def load_and_split_data(directory, validation_split=0.2, test_split=0.1, seed=123):
    # Load the entire dataset
    full_dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        seed=seed,
        shuffle=True,
        label_mode='categorical',
        image_size=(224, 224),
        batch_size=32) 

    # Calculate the number of batches needed for each split
    total_batches = len(full_dataset)
    val_batches = int(total_batches * validation_split)
    test_batches = int(total_batches * test_split)
    train_batches = total_batches - val_batches - test_batches

    # Split the dataset into train, validation, and test
    train_dataset = full_dataset.take(train_batches)
    test_dataset = full_dataset.skip(train_batches).take(test_batches)
    validation_dataset = full_dataset.skip(train_batches + test_batches)

    return train_dataset, validation_dataset, test_dataset


In [14]:
base_dir = 'proccesed_dataset'
train_ds, val_ds, test_ds = load_and_split_data(base_dir)

Found 8303 files belonging to 6 classes.


In [15]:
def make_model(input_shape, num_classes):
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2)
    ])
    
    base_model = MobileNetV2(input_shape=input_shape,
                            include_top=False,
                            weights='imagenet')
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False
    
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = base_model(inputs, training=True)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    
    # Increased L2 regularization for weight decay effect
    x = Dense(2048, activation='relu', 
              kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.7)(x)
    
    x = Dense(1024, activation='relu', 
              kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.6)(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

In [16]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

In [17]:
model = make_model(input_shape=(224, 224, 3), num_classes=6)
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=5e-5
    ),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [18]:
for images, labels in train_ds.take(1):
    print(images.shape)  # Should be (batch_size, 224, 224, 3)
    print(labels.shape)  # Should be (batch_size, 6)

(32, 224, 224, 3)
(32, 6)


In [19]:
class AccuracyThreshold(Callback):
    def __init__(self, threshold=0.85):
        super(AccuracyThreshold, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get("val_accuracy")
        accuracy = logs.get("accuracy")
        if val_accuracy is not None:
            if val_accuracy >= self.threshold:
                print(f"\nReached {self.threshold * 100}% val accuracy. Stopping training...")
                self.model.stop_training = True
            elif accuracy >= 0.95:
                print(f"\nReached 95% accuracy. Stopping training...")
                self.model.stop_training = True

# Instantiate the custom callback with a threshold of 85% (0.85)
accuracy_threshold_callback = AccuracyThreshold(threshold=0.85)

# Early Stopping and Model Checkpoint
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=15,  # More patience
    restore_best_weights=True,
    min_delta=0.001  # Minimum change to qualify as an improvement
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False
)

# Learning Rate Reducer
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Reduce learning rate by half
    patience=5,
    min_lr=1e-6
)

In [20]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[accuracy_threshold_callback,early_stopping,model_checkpoint,reduce_lr] 
)

Epoch 1/50
182/182 [==============================] - 55s 283ms/step - loss: 31.4689 - accuracy: 0.3350 - val_loss: 29.9277 - val_accuracy: 0.4918 - lr: 5.0000e-05
Epoch 2/50
182/182 [==============================] - 48s 264ms/step - loss: 30.0395 - accuracy: 0.4722 - val_loss: 28.9144 - val_accuracy: 0.5883 - lr: 5.0000e-05
Epoch 3/50
182/182 [==============================] - 49s 269ms/step - loss: 28.9767 - accuracy: 0.5319 - val_loss: 28.0117 - val_accuracy: 0.6351 - lr: 5.0000e-05
Epoch 4/50
182/182 [==============================] - 48s 264ms/step - loss: 27.9596 - accuracy: 0.5707 - val_loss: 27.1412 - val_accuracy: 0.6521 - lr: 5.0000e-05
Epoch 5/50
182/182 [==============================] - 51s 281ms/step - loss: 26.9020 - accuracy: 0.6198 - val_loss: 26.2187 - val_accuracy: 0.6563 - lr: 5.0000e-05
Epoch 6/50
182/182 [==============================] - 48s 264ms/step - loss: 25.9560 - accuracy: 0.6386 - val_loss: 25.3404 - val_accuracy: 0.6618 - lr: 5.0000e-05
Epoch 7/50
182/1

In [21]:
# Assuming test_ds is your test dataset
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


26/26 [==============================] - 5s 164ms/step - loss: 11.1928 - accuracy: 0.7656
Test accuracy: 0.765625
Test loss: 11.19279670715332


In [22]:
model.save('model//MobileNetModelV2')  # Saves to HDF5 file (requires h5py installed)


INFO:tensorflow:Assets written to: model//MobileNetModelV2\assets


INFO:tensorflow:Assets written to: model//MobileNetModelV2\assets


In [23]:
import pandas as pd

In [24]:
# Convert to DataFrame
history_df = pd.DataFrame(history.history)

# Save to CSV
history_df.to_csv('model_history2.csv', index=False)

In [25]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

In [26]:
def extract_labels(dataset):
    labels_list = []
    for _, labels in dataset:
        labels_list.append(labels.numpy())
    return np.concatenate(labels_list, axis=0)

val_labels = extract_labels(val_ds)

In [27]:

# After training
y_pred = model.predict(val_ds)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(val_labels, axis=1)

print(confusion_matrix(y_true_classes, y_pred_classes))
print(classification_report(y_true_classes, y_pred_classes))

52/52 [==============================] - 10s 160ms/step
[[48 34 40 31 50 40]
 [46 55 56 37 58 64]
 [46 49 60 33 56 52]
 [40 25 41 25 28 36]
 [46 50 67 43 57 52]
 [35 53 50 27 63 54]]
              precision    recall  f1-score   support

           0       0.18      0.20      0.19       243
           1       0.21      0.17      0.19       316
           2       0.19      0.20      0.20       296
           3       0.13      0.13      0.13       195
           4       0.18      0.18      0.18       315
           5       0.18      0.19      0.19       282

    accuracy                           0.18      1647
   macro avg       0.18      0.18      0.18      1647
weighted avg       0.18      0.18      0.18      1647



In [28]:
import matplotlib.pyplot as plt